# MIT-BIH Arrhythmia

En esta notebook, implementaremos técnicas de regularización, tales como early stopping y dropout, utilizando el [MIT-BIH Arrhythmia Database](https://physionet.org/content/mitdb/1.0.0). El objetivo principal es mejorar la generalización de nuestros modelos de aprendizaje profundo y prevenir el sobreajuste durante el entrenamiento.

## Introducción

### Objetivos

1. **Implementar y entender las técnicas de regularización** más comunes en redes neuronales, incluyendo early stopping y dropout.
2. **Entrenar modelos de clasificación** en PyTorch utilizando el MIT-BIH Arrhythmia Database.
3. **Evaluar el impacto de las técnicas de regularización** en el rendimiento del modelo utilizando métricas adecuadas.

### Contenido

1. Configuración de bibliotecas y semillas para reproducibilidad.
2. Carga y exploración del MIT-BIH Arrhythmia Database.
3. Preparación de los datos y división en conjuntos de entrenamiento y prueba.
4. Definición y entrenamiento de modelos de clasificación en PyTorch.
5. Implementación de técnicas de regularización como dropout y early stopping.
6. Evaluación y comparación del rendimiento de los modelos con y sin regularización.

### Sobre el conjunto de datos

El MIT-BIH Arrhythmia Database es un conjunto de datos ampliamente utilizado en la investigación de la arritmia cardíaca. Contiene registros de electrocardiogramas (ECG) de diferentes pacientes, con anotaciones detalladas sobre distintos tipos de arritmias. Este conjunto de datos nos permitirá entrenar modelos de clasificación capaces de identificar distintos tipos de arritmias a partir de los datos de ECG, haciendo énfasis en la importancia de la regularización para mejorar la precisión y generalización de los modelos.

El modelo esta disponible en el siguiente [link](https://www.kaggle.com/shayanfazeli/heartbeat).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from torchinfo import summary

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    classification_report,
)

from utils import plot_taining

In [ ]:
# Fijamos la semilla para que los resultados sean reproducibles
SEED = 34

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
import sys

# definimos el dispositivo que vamos a usar
DEVICE = "cpu"  # por defecto, usamos la CPU
if torch.cuda.is_available():
    DEVICE = "cuda"  # si hay GPU, usamos la GPU
elif torch.backends.mps.is_available():
    DEVICE = "mps"  # si no hay GPU, pero hay MPS, usamos MPS
elif torch.xpu.is_available():
    DEVICE = "xpu"  # si no hay GPU, pero hay XPU, usamos XPU

print(f"Usando {DEVICE}")

NUM_WORKERS = 0 # Win y MacOS pueden tener problemas con múltiples workers
if sys.platform == 'linux':
    NUM_WORKERS = 4  # numero de workers para cargar los datos (depende de cada caso)

print(f"Usando {NUM_WORKERS}")

In [ ]:
BATCH_SIZE = 2048  # tamaño del batch

## Carga de datos + Exploración

In [ ]:
TRAIN_DATA_PATH = "data/mitbih_train.csv"
TEST_DATA_PATH = "data/mitbih_test.csv"

In [ ]:
df_train = pd.read_csv(TRAIN_DATA_PATH, header=None)
df_test = pd.read_csv(TEST_DATA_PATH, header=None)

# Concatenamos los datos de entrenamiento y test
df = pd.concat([df_train, df_test], axis=0)

In [ ]:
ninputs = df.shape[1] - 1
nclasses = df.iloc[:, -1].nunique()
print(f"Existen {nclasses} clases y {ninputs} características")

In [ ]:
df_train.info()
df_train.head()

In [ ]:
df_test.info()
df_test.head()

In [ ]:
X_train = df_train.iloc[:, :-1]  # Extraemos las características
y_train = df_train.iloc[:, -1]  # Extraemos las etiquetas

X_test = df_test.iloc[:, :-1]
y_test = df_test.iloc[:, -1]

In [ ]:
TARGET_NAMES = [
    "Normal beat",
    "Supraventricular premature beat",
    "Premature ventricular contraction",
    "Fusion of ventricular and normal beat",
    "Unclassifiable beat",
]

### Distribución de clases

Veamos la distribución de las clases en el conjunto de datos para comprender mejor el problema de clasificación que estamos abordando.

In [ ]:
class_count = df.iloc[:, -1].value_counts()
print(class_count)

class_count.sort_index().plot(kind="bar", title="Número de muestras por clase")

Como se puede observar, las clases están desbalanceadas, lo que puede afectar el rendimiento de nuestro modelo. Por lo tanto, es importante tener en cuenta este desbalance al evaluar el rendimiento del modelo y considerar estrategias como el uso de pesos de clase o técnicas de aumento de datos.

## Datasets y Dataloaders

### Dataset

Vamos a crear un `Dataset` personalizado para cargar y preprocesar los datos del MIT-BIH Arrhythmia Database. Este `Dataset` se encargará de cargar los datos desde nuestros dataframe de Pandas, aplicar transformaciones, y devolver los datos en el formato adecuado para ser procesados por nuestros modelos de PyTorch.

In [ ]:
class MITBIHDataSet(Dataset):
    def __init__(self, df_features, df_target, num_classes):
        self.x_df = df_features.values
        self.y_df = df_target.values
        self.num_classes = num_classes

    def __len__(self):
        return len(self.x_df)

    def __getitem__(self, idx):
        x = torch.tensor(self.x_df[idx], dtype=torch.float32)
        y = torch.tensor(self.y_df[idx], dtype=torch.long)
        return x, y

### Split de datos

Vamos a dividir nuestro conjunto de datos en conjuntos de entrenamiento y validacion. Esta vez en vez de hacerlo con [torch.utils.data.random_split](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) lo haremos con [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) para poder mantener el balance de clases en ambos conjuntos.

> Por lo que pudimos observar en la exploración de datos, el conjunto de prueba ya tiene un balance de clases similar al conjunto de entrenamiento.

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=SEED, stratify=y_train
)

In [ ]:
# Contar la frecuencia de cada clase en cada conjunto
class_counts_train = y_train.value_counts()
class_counts_val = y_val.value_counts()

# Crear subplots para los histogramas
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Histograma para train
axes[0].bar(class_counts_train.index, class_counts_train.values, color="skyblue")
axes[0].set_title("Distribución de Clases en trian")
axes[0].set_xlabel("Clases")
axes[0].set_ylabel("Frecuencia")

# Histograma para val
axes[1].bar(class_counts_val.index, class_counts_val.values, color="lightgreen")
axes[1].set_title("Distribución de Clases en val")
axes[1].set_xlabel("Clases")
axes[1].set_ylabel("Frecuencia")

plt.show()

In [ ]:
train_dataset = MITBIHDataSet(X_train, y_train, nclasses)
val_dataset = MITBIHDataSet(X_val, y_val, nclasses)
test_dataset = MITBIHDataSet(X_test, y_test, nclasses)

### DataLoaders

Definimos los dataloaders para cada conjunto de datos, estos son los que se encargan de cargar los datos en lotes durante el entrenamiento y la evaluación del modelo.

In [ ]:
def get_data_loaders(batch_size):

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS
    )

    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS
    )

    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS
    )

    return train_loader, val_loader, test_loader

In [ ]:
train_loader, val_loader, test_loader = get_data_loaders(
    BATCH_SIZE
)  # obtenemos los dataloaders

# probamos un batch del DataLoader
x_batch, y_batch = next(iter(train_loader))
print(x_batch.shape, y_batch.shape)

## Definición de modelo

Empezaremos definiendo un modelo de clasificación simple en PyTorch. Este modelo constará de dos capas.

In [ ]:
# MLP                                      [2048, 5]
# ├─Linear: 1-1                            [2048, 512]
# ├─Linear: 1-2                            [2048, 2048]
# ├─Linear: 1-3                            [2048, 5]

class MLP(nn.Module):
    def __init__(self, input_size, nclass):
        super(MLP, self).__init__()
        pass

    def forward(self, x):
        pass
        return x

summary(MLP(ninputs, nclasses), input_size=(BATCH_SIZE, ninputs))

## Entrenamiento

In [ ]:
def evaluate(model, criterion, data_loader, device=DEVICE):
    """
    Evalúa el modelo en los datos proporcionados y calcula la pérdida promedio.

    Args:
        model (torch.nn.Module): El modelo que se va a evaluar.
        criterion (torch.nn.Module): La función de pérdida que se utilizará para calcular la pérdida.
        data_loader (torch.utils.data.DataLoader): DataLoader que proporciona los datos de evaluación.

    Returns:
        float: La pérdida promedio en el conjunto de datos de evaluación.

    """
    model.eval()  # ponemos el modelo en modo de evaluacion
    total_loss = 0 # acumulador de la perdida
    with torch.no_grad(): # deshabilitamos el calculo de gradientes
        for x, y in data_loader: # iteramos sobre el dataloader
            x = x.to(DEVICE) # movemos los datos al dispositivo
            y = y.to(DEVICE) # movemos los datos al dispositivo
            output = model(x) # forward pass
            total_loss += criterion(output, y).item() # acumulamos la perdida
    return total_loss / len(data_loader) # retornamos la perdida promedio


def train(
    model,
    optimizer,
    criterion,
    train_loader,
    val_loader,
    device=DEVICE,
    epochs=10,
    log_fn=None,
    log_every=1,
):
    """
    Entrena el modelo utilizando el optimizador y la función de pérdida proporcionados.

    Args:
        model (torch.nn.Module): El modelo que se va a entrenar.
        optimizer (torch.optim.Optimizer): El optimizador que se utilizará para actualizar los pesos del modelo.
        criterion (torch.nn.Module): La función de pérdida que se utilizará para calcular la pérdida.
        train_loader (torch.utils.data.DataLoader): DataLoader que proporciona los datos de entrenamiento.
        val_loader (torch.utils.data.DataLoader): DataLoader que proporciona los datos de validación.
        epochs (int): Número de épocas de entrenamiento (default: 10).
        log_fn (function): Función que se llamará después de cada log_every épocas con los argumentos (epoch, train_loss, val_loss) (default: None).
        log_every (int): Número de épocas entre cada llamada a log_fn (default: 1).

    Returns:
        Tuple[List[float], List[float]]: Una tupla con dos listas, la primera con el error de entrenamiento de cada época y la segunda con el error de validación de cada época.

    """
    epoch_train_errors = []  # colectamos el error de traing para posterior analisis
    epoch_val_errors = []  # colectamos el error de validacion para posterior analisis

    for epoch in range(epochs): # loop de entrenamiento
        model.train()  # ponemos el modelo en modo de entrenamiento
        train_loss = 0 # acumulador de la perdida de entrenamiento
        for x, y in train_loader:
            x = x.to(device) # movemos los datos al dispositivo
            y = y.to(device) # movemos los datos al dispositivo

            optimizer.zero_grad() # reseteamos los gradientes

            output = model(x) # forward pass (prediccion)
            batch_loss = criterion(output, y) # calculamos la perdida con la salida esperada

            batch_loss.backward() # backpropagation
            optimizer.step() # actualizamos los pesos

            train_loss += batch_loss.item() # acumulamos la perdida

        train_loss /= len(train_loader) # calculamos la perdida promedio de la epoca
        epoch_train_errors.append(train_loss) # guardamos la perdida de entrenamiento
        val_loss = evaluate(model, criterion, val_loader) # evaluamos el modelo en el conjunto de validacion
        epoch_val_errors.append(val_loss) # guardamos la perdida de validacion

        if log_fn is not None: # si se pasa una funcion de log
            if (epoch + 1) % log_every == 0: # loggeamos cada log_every epocas
                log_fn(epoch, train_loss, val_loss) # llamamos a la funcion de log

    return epoch_train_errors, epoch_val_errors

In [ ]:
LR = 0.01
CRITERION = nn.CrossEntropyLoss().to(DEVICE)
EPOCHS = 200

In [ ]:
def print_log(epoch, train_loss, val_loss):
    print(
        f"Epoch: {epoch + 1:03d}/{EPOCHS:03d} | Train Loss: {train_loss:.5f} | Val Loss: {val_loss:.5f}"
    )

base_model = MLP(ninputs, nclasses).to(DEVICE)

optimizer = optim.Adam(base_model.parameters(), lr=LR)

epoch_train_errors, epoch_val_errors = train(
    base_model,
    optimizer,
    CRITERION,
    train_loader,
    val_loader,
    DEVICE,
    EPOCHS,
    print_log,
    5,
)

## Loss durante el entrenamiento

In [ ]:
# graficamos los errores de entrenamiento y validación
plot_taining(epoch_train_errors, epoch_val_errors)
# graficamos los errores de entrenamiento y validación a partir de la época 5
plot_taining(epoch_train_errors[5:], epoch_val_errors[5:])

## Evaluación

Vamos a evaluar el rendimiento de nuestro modelo en el conjunto de prueba utilizando métricas como la precisión, el recall y la matriz de confusión.

In [ ]:
def model_classification_report(model, dataloader):
    # Evaluación del modelo
    model.eval()

    all_preds = []
    all_labels = []

    pass # TODO

    # Calcular precisión (accuracy)
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Accuracy: {accuracy:.4f}\n")

    # Reporte de clasificación
    report = classification_report(
        all_labels, all_preds, target_names=TARGET_NAMES
    )
    print("Reporte de clasificación:\n", report)

In [ ]:
model_classification_report(base_model, test_loader)

**Métricas por Clase**
- **Precision:** De lo que predije como esta clase, ¿cuánto acerté? (↑ = menos falsos positivos)
  - `Precision = TP / (TP + FP)`
- **Recall:** De todos los casos reales, ¿cuántos detecté? (↑ = menos falsos negativos)  
  - `Recall = TP / (TP + FN)`
- **F1-Score:** Balance entre precision y recall (media armónica)
  - `F1 = 2 * (Precision * Recall) / (Precision + Recall)`
- **Support:** Cantidad de muestras reales de esa clase

**Métricas Globales**
- **Accuracy:** % total de aciertos `(TP + TN) / Total`
- **Macro Avg:** Promedio simple (todas las clases pesan igual)
- **Weighted Avg:** Promedio ponderado por support (refleja desbalance)

**Interpretación Rápida**
- Valores → 1.0 = Mejor | Valores → 0.0 = Peor
- Precision > Recall = Modelo conservador
- Recall > Precision = Modelo agresivo  
- Macro < Weighted = Mejor en clases mayoritarias
- Support desigual = Dataset desbalanceado

> **Donde:** TP = True Positives, FP = False Positives, FN = False Negatives, TN = True Negatives

## Ejercicios

1. **Implementar dropout**: Agregar más capas lineales al modelo y aplicar dropout después de cada capa. Evaluar el impacto de dropout en el rendimiento del modelo.

2. **Implementar early stopping**: Implementar la técnica de early stopping para detener el entrenamiento cuando el rendimiento del modelo deja de mejorar en el conjunto de validación. Evaluar el impacto de early stopping en el rendimiento del modelo.

3. **Implementar label smoothing**: Implementar la técnica de label smoothing para mejorar la generalización del modelo. Evaluar el impacto de label smoothing en el rendimiento del modelo.

4. **Modelo libre**: Implementar un modelo de clasificación en PyTorch utilizando el MIT-BIH Arrhythmia Database. Experimentar con diferentes arquitecturas de modelos, hiperparámetros y técnicas de regularización para mejorar el rendimiento del modelo.

5. **Comparación de modelos**: Con cuál de los modelos te quedarías si: 
    - Tuvieras que elegir el modelo con mejor precisión en promedio.
    - Tuvieras que elegir el modelo con mejor recall en promedio.
    - Tuvieras que elegir el modelo con mejor F1-score en promedio.

### Ejericio 1: Implementar dropout

[Dropout](https://jmlr.org/papers/v15/srivastava14a.html) es una técnica de regularización utilizada en redes neuronales para prevenir el sobreajuste (overfitting). Consiste en desactivar aleatoriamente un subconjunto de neuronas durante el entrenamiento en cada paso de propagación hacia adelante. Esto ayuda a la red a no depender demasiado de ninguna neurona en particular, promoviendo una representación más robusta y generalizable de los datos.

**¿Cómo Funciona Dropout?**

Durante el entrenamiento, Dropout:
1. **Apagado Aleatorio de Neuronas**: Un porcentaje de neuronas se apaga aleatoriamente en cada iteración.
2. **Escalado de Neuronas Activas**: Las activaciones de las neuronas restantes se escalan para mantener el equilibrio en la red.

**Evaluación y Dropout**

Durante la evaluación, es crucial que las capas Dropout se comporten de manera diferente:
- **Entrenamiento (`model.train()`)**: Dropout está activo, apagando neuronas y escalando las activaciones.
- **Evaluación (`model.eval()`)**: Dropout está desactivado, permitiendo que todas las neuronas estén activas sin escalado de activaciones. Esto asegura que el modelo utilice toda su capacidad para hacer predicciones.

**Implementación de Dropout en PyTorch**

En PyTorch, podemos implementar Dropout utilizando la capa [`torch.nn.Dropout`](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html). Esta capa apaga aleatoriamente un porcentaje de neuronas durante el entrenamiento y escala las activaciones durante la evaluación.

In [ ]:
rand_torch = torch.rand((2,5)) # genero datos al azar
print(rand_torch) # imprimimos sus valores
drop_layer = nn.Dropout(0.5) # creamos una capa de dropout con un 50% de apagar una neurona

In [ ]:
drop_layer.train() # por defecto ya esta en train mode
print(drop_layer(rand_torch)) # 1) apaga (pone en 0) con una probablidad p definida en la capa 2) pondera los valores restantes proporcionalmente

In [ ]:
drop_layer.eval() # al ponerlo en eval no deberia surtir efecto al pasar por la capa
print(drop_layer(rand_torch))

Algunas reglas prácticas para usar Dropout de manera efectiva:

- **Ubicación: "Después de Activación, Nunca en Output"**
```python
x = F.relu(self.fc(x))
x = self.dropout(x)  # ✅ Después de activación
# ❌ NUNCA dropout en la capa final
```

- **Valores: "20-50 Rule"**
```python
dropout = nn.Dropout(0.5)    # Capas ocultas (default)
dropout = nn.Dropout(0.2)    # CNNs, RNNs, entrada
# Si dudas → usa 0.3
```

- **Train/Eval: "Siempre Cambia Modo"**
```python
model.train()   # Entrenamiento
model.eval()    # ⚠️ CRÍTICO para inferencia
```

- **Cuándo Usar: "Solo si Overfittea"**
```python
# train_acc=99%, val_acc=85% → Agrega dropout
# train_acc=80%, val_acc=78% → No necesitas
# Empieza sin dropout, agrega si necesitas
```

- **Arquitectura: "Más Profundo = Menos Dropout"**
```python
# 2-3 capas:  p=0.5
# 5-10 capas: p=0.2-0.3  
# 10+ capas:  p=0.1 o nada
```

In [ ]:
class MLP_EJ1(nn.Module):
    def __init__(self, input_size, nclass, dropout=0.5):
        super(MLP_EJ1, self).__init__()
        pass

    def forward(self, x):
        pass

summary(MLP_EJ1(ninputs, nclasses), input_size=(BATCH_SIZE, ninputs))

Algunas preguntas para pensar:
- ¿Por qué dropout no tiene parametros entrenables?
- ¿Qué pasaría si no ponemos el modelo en eval mode durante la evaluación?

In [ ]:
ej1_model = MLP_EJ1(ninputs, nclasses, 0.2).to(DEVICE)

optimizer = optim.Adam(ej1_model.parameters(), lr=LR)

epoch_train_errors, epoch_val_errors = train(
    ej1_model,
    optimizer,
    CRITERION,
    train_loader,
    val_loader,
    DEVICE,
    EPOCHS,
    print_log,
    5,
)

In [ ]:
# graficamos los errores de entrenamiento y validación
plot_taining(epoch_train_errors, epoch_val_errors)
# graficamos los errores de entrenamiento y validación a partir de la época 5
plot_taining(epoch_train_errors[5:], epoch_val_errors[5:])

In [ ]:
model_classification_report(ej1_model, test_loader)

- ¿Cómo se compara el resultado con el modelo sin dropout?

#### Lectura Adicional
- [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](https://jmlr.org/papers/v15/srivastava14a.html) - Paper original de Dropout
- [Deep Learning Book - Regularization](https://www.deeplearningbook.org/contents/regularization.html)
- [Dive into Deep Learning - Dropout](https://d2l.ai/chapter_multilayer-perceptrons/dropout.html)

### Ejercicio 2: Implementar early stopping

[Early stopping](https://en.wikipedia.org/wiki/Early_stopping) es una técnica de regularización utilizada para prevenir el sobreajuste en modelos de aprendizaje profundo. Consiste en detener el entrenamiento del modelo cuando el rendimiento en el conjunto de validación deja de mejorar, evitando así que el modelo se ajuste demasiado a los datos de entrenamiento.

**¿Cómo Funciona Early Stopping?**

Early stopping se basa en el principio de que, a medida que el modelo se entrena, el rendimiento en el conjunto de validación debería mejorar hasta cierto punto y luego comenzar a empeorar. Esto se debe a que el modelo se ajusta cada vez más a los datos de entrenamiento, lo que puede llevar a un sobreajuste. Early stopping busca detener el entrenamiento en el punto óptimo, antes de que el modelo comience a sobreajustarse.

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5):
        """
        Args:
            patience (int): Cuántas épocas esperar después de la última mejora.
        """
        pass

    def __call__(self, val_loss):
        pass

Nuestra clase `EarlyStopping` implementa la lógica de early stopping muy sencilla. Guarda la mejor loss de validación vista hasta el momento y detiene el entrenamiento si la loss de validación no mejora después de un cierto número de épocas.

In [ ]:
# lo ponemos a prueba con un ejemplo sencillo
early_stopping = EarlyStopping(patience=2)
loss_simulated = [0.1, 0.09, 0.08, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15]

for i, loss in enumerate(loss_simulated):
    early_stopping(loss)
    if early_stopping.early_stop:
        print(
            f"Detener entrenamiento en la época {i + 1}, la mejor pérdida fue {early_stopping.best_score}"
        )
        break

Vamos a redefinir nuestro training loop para incluir la lógica de early stopping.

In [ ]:
def train_es(
    model,
    optimizer,
    criterion,
    train_loader,
    val_loader,
    device,
    patience=5,
    epochs=10,
    log_fn=None,
    log_every=1,
):
    """
    Entrena el modelo utilizando el optimizador y la función de pérdida proporcionados.

    Args:
        model (torch.nn.Module): El modelo que se va a entrenar.
        optimizer (torch.optim.Optimizer): El optimizador que se utilizará para actualizar los pesos del modelo.
        criterion (torch.nn.Module): La función de pérdida que se utilizará para calcular la pérdida.
        train_loader (torch.utils.data.DataLoader): DataLoader que proporciona los datos de entrenamiento.
        val_loader (torch.utils.data.DataLoader): DataLoader que proporciona los datos de validación.
        device (str): El dispositivo donde se ejecutará el entrenamiento.
        patience (int): Número de épocas a esperar después de la última mejora en val_loss antes de detener el entrenamiento (default: 5).
        epochs (int): Número de épocas de entrenamiento (default: 10).
        log_fn (function): Función que se llamará después de cada log_every épocas con los argumentos (epoch, train_loss, val_loss) (default: None).
        log_every (int): Número de épocas entre cada llamada a log_fn (default: 1).

    Returns:
        Tuple[List[float], List[float]]: Una tupla con dos listas, la primera con el error de entrenamiento de cada época y la segunda con el error de validación de cada época.

    """
    pass # TODO

In [ ]:
ej2_model = MLP(ninputs, nclasses).to(DEVICE)

optimizer = optim.Adam(ej2_model.parameters(), lr=LR)

epoch_train_errors, epoch_val_errors = train_es(
    ej2_model,
    optimizer,
    CRITERION,
    train_loader,
    val_loader,
    DEVICE,
    3,
    EPOCHS,
    print_log,
    5,
)

In [ ]:
# graficamos los errores de entrenamiento y validación
plot_taining(epoch_train_errors, epoch_val_errors)
# graficamos los errores de entrenamiento y validación a partir de la época 5
plot_taining(epoch_train_errors[5:], epoch_val_errors[5:])

In [ ]:
model_classification_report(ej2_model, test_loader)

#### Lectura Adicional
- "Other than the obvious difference with the previous study (this used real data), it is
important to note another significant point: in this case. we stopped iterating (by anyone
particular criterion) when that criterion was leading to no new test set performance
improvement" - From: [Generalization and Parameter Estimation in Feedforward Nets: Some Experiments](https://proceedings.neurips.cc/paper/1989/file/63923f49e5241343aa7acb6a06a751e7-Paper.pdf)
- [Deep Learning Book - Regularization](https://www.deeplearningbook.org/contents/regularization.html)
- [Dive into Deep Learning - Early Stopping](https://d2l.ai/chapter_multilayer-perceptrons/generalization-deep.html#early-stopping)

### Ejercicio 3: Implementar label smoothing

[Label smoothing](https://towardsdatascience.com/what-is-label-smoothing-108debd7ef06) es una técnica de regularización utilizada en el entrenamiento de modelos de clasificación para mejorar la generalización y evitar que el modelo se vuelva demasiado confiado en sus predicciones. Consiste en suavizar las etiquetas de clase, en lugar de asignar una probabilidad de 1 a la clase correcta y 0 a las demás, se asigna una probabilidad ligeramente menor a la clase correcta y una probabilidad ligeramente mayor a las demás clases.

Se pude aplicar con la siguiente formula:

$$ y_{ls} = (1 - \alpha) \cdot y_{hot} + \frac{\alpha}{K} $$

donde:
- $y_{ls}$ son las etiquetas suavizadas.
- $y_{hot}$ son las etiquetas originales en formato one-hot.
- $\alpha$ es el factor de suavizado (entre 0 y 1).
- $K$ es el número de clases.

In [ ]:
# Ejemplo de etiquetas (índices de clase)
z = torch.tensor([0, 1, 2, 1, 3, 2])  # Etiquetas de ejemplo
print(f"Etiquetas\n{z}")

# Convertir a one-hot encoding
z_one_hot = F.one_hot(z, nclasses)
print(f"\nOne-hot encoding\n{z_one_hot}")

# Parámetro de suavizado
alpha = 0.1  # Factor de suavizado
K = nclasses

# Aplicar label smoothing
z_ls = (1 - alpha) * z_one_hot + alpha / K
print(f"\nLabel smoothing\n{z_ls}")

Afortunadamente, PyTorch proporciona una implementación de label smoothing en la función de pérdida [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) a través del parámetro `label_smoothing`. Al establecer `label_smoothing` en un valor mayor que cero, se aplica label smoothing a la función de pérdida.

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1).to(DEVICE)

In [ ]:
ej3_model = MLP(ninputs, nclasses).to(DEVICE)

optimizer = optim.Adam(ej3_model.parameters(), lr=LR)

epoch_train_errors, epoch_val_errors = train(
    ej3_model,
    optimizer,
    criterion,
    train_loader,
    val_loader,
    DEVICE,
    EPOCHS,
    print_log,
    5,
)

In [ ]:
model_classification_report(ej3_model, test_loader)